In [1]:
from root_finding import *

In [ ]:

import os
import numpy as np
from pathlib import Path
from tqdm import trange

SAVE_DIR = Path("./saved_trees_trials")
SAVE_DIR.mkdir(exist_ok=True)

SEED_FILE = SAVE_DIR / "global_seed_counter.txt"

def load_seed():
    if not SEED_FILE.exists():
        SEED_FILE.write_text("0")
        return 0
    return int(SEED_FILE.read_text().strip())

def update_seed(k):
    SEED_FILE.write_text(str(k))

global_seed = load_seed()

TREE_SIZES = [125000]#[30000,40000,75000]#[100,200,500,1000,2000,5000,10000,20000,50000, 100000]  # large trees
NUM_TRIALS = 17  # adjust as needed

for n in TREE_SIZES:
    print(f"\nGenerating trees of size n={n}...")
    tree_dir = SAVE_DIR / f"torus_trees_n={n}_trials"
    tree_dir.mkdir(exist_ok=True)

    # Determine the next trial index
    existing_trials = sorted(tree_dir.glob("trial_*"))
    if existing_trials:
        # Extract last index
        last_idx = max(int(trial.name.split("_")[1]) for trial in existing_trials)
        start_t = last_idx + 1
    else:
        start_t = 0

    end_t = start_t + NUM_TRIALS

    for t in trange(start_t, end_t, desc=f"n={n}"):
        parents, points = nearest_attachment_tree_blockwise_gpu(n)

        trial_dir = tree_dir / f"trial_{t:06d}"
        trial_dir.mkdir(exist_ok=True)
        np.save(trial_dir / "parents.npy", parents)
        np.save(trial_dir / "points.npy", points)

print("Done!")
print(f"Files saved in: {SAVE_DIR}")



Generating trees of size n=125000...


n=125000:   0%|          | 0/17 [00:00<?, ?it/s]

n=125000: 100%|██████████| 17/17 [01:37<00:00,  5.76s/it]

Done!
Files saved in: saved_trees_trials


In [ ]:

import os
import numpy as np
from pathlib import Path
from tqdm import trange

d = 3
alpha = 0.0
beta = 50.0


# New save directory for *other models*
BASE_SAVE_DIR = Path("./saved_other_models")
BASE_SAVE_DIR.mkdir(exist_ok=True)

# Folder naming convention: d=3_alpha=1.0_beta=2.0
MODEL_DIR = BASE_SAVE_DIR / f"d={d}_alpha={alpha}_beta={beta}"
MODEL_DIR.mkdir(exist_ok=True)

SEED_FILE = MODEL_DIR / "global_seed_counter.txt"


# ---------- Global seed handling ----------
def load_seed():
    if not SEED_FILE.exists():
        SEED_FILE.write_text("0")
        return 0
    return int(SEED_FILE.read_text().strip())

def update_seed(k):
    SEED_FILE.write_text(str(k))

global_seed = load_seed()


TREE_SIZES = [100,200,500,1000,2000,5000,10000]  # change as needed
NUM_TRIALS = 100      # adjust as needed

for n in TREE_SIZES:
    print(f"\nGenerating trees of size n={n}...")
    
    # Normal file structure: <MODEL_DIR>/trees_n=XXXXXX_trials/
    tree_dir = MODEL_DIR / f"trees_n={n}_trials"
    tree_dir.mkdir(exist_ok=True)

    # Find the next trial index
    existing_trials = sorted(tree_dir.glob("trial_*"))
    if existing_trials:
        last_idx = max(int(trial.name.split("_")[1]) for trial in existing_trials)
        start_t = last_idx + 1
    else:
        start_t = 0

    end_t = start_t + NUM_TRIALS

    for t in trange(start_t, end_t, desc=f"n={n}"):

        # *** NEW MODEL ***
        parents, points = attachment_tree_alpha_beta_blockwise_gpu(
            n, d=d, alpha=alpha, beta=beta
        )

        trial_dir = tree_dir / f"trial_{t:06d}"
        trial_dir.mkdir(exist_ok=True)

        np.save(trial_dir / "parents.npy", parents)
        np.save(trial_dir / "points.npy", points)

        # Update seed *per trial* if needed
        global_seed += 1
        update_seed(global_seed)

print("Done!")
print(f"Files saved in: {MODEL_DIR}")



Generating trees of size n=100...


n=100: 100%|██████████| 100/100 [00:00<00:00, 137.19it/s]



Generating trees of size n=200...


n=200: 100%|██████████| 100/100 [00:00<00:00, 132.98it/s]



Generating trees of size n=500...


...


Generating trees of size n=5000...


n=5000: 100%|██████████| 100/100 [00:09<00:00, 10.32it/s]



Generating trees of size n=10000...


n=10000: 100%|██████████| 100/100 [00:39<00:00,  2.52it/s]

Done!
Files saved in: saved_other_models\d=3_alpha=0.0_beta=50.0
